# 数据挖掘Q3Q4
> 1651718 方沛

In [1]:
import pandas as pd
import numpy as np
import datetime
import torch
import math
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
import copy
import torch.utils.data as da
from sklearn.neural_network import MLPRegressor
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA 

对数据集进行预处理,回复时间字符串格式为datetime,同时将pluno类型转化为字符串

In [23]:
# 数据预处理
def read_dataset(path):
    dataset = pd.read_csv(path)
    dataset['purchase_date']=[x.replace('/','-') for x in dataset["purchase_date"]]
    dataset['purchase_date']=pd.to_datetime(dataset['purchase_date'])
    dataset['purchase_date']=[datetime.datetime.strftime(x,'%Y-%m-%d') for x in dataset['purchase_date']]
    dataset['pluno'] = dataset['pluno'].astype('str')
    
    plunos = np.unique(dataset['pluno'])
    dates = np.unique(dataset['purchase_date'])
    train_dates = dates
    test_dates = dates[146:]
    
    for i in range(1,5):
        # 训练一个pca
        col = "pl_"+str(i)
        record = dict(dataset.loc[dataset["purchase_date"].isin(train_dates)].groupby(by=[col])['qty'].mean())
        dataset[col]=[round(record[x],2) for x in dataset[col]]
        
        # one-hot
        enc=OneHotEncoder(categories='auto')
        enc.fit(np.array(dataset[col]).reshape(-1,1))
        dplunos = enc.transform(np.array(dataset[col]).reshape(-1,1)).toarray()

        # pca
        pca=PCA(n_components=1)
        pca.fit(dplunos)
        compressed_dplunos = pca.transform(dplunos)
        compressed_dplunos = np.around(compressed_dplunos, decimals=2)
        
        dataset[col] =compressed_dplunos[:,0]
        for c in range(1,compressed_dplunos.shape[1]):
            new_pl = col+'_'+str(c)
            col_name = dataset.columns.tolist()
            col_name.insert(col_name.index(col)+1,new_pl)
            dataset=dataset.reindex(columns=col_name)
            dataset[new_pl] = compressed_dplunos[:,c]
            
        


    return dataset.fillna(0)

dataset = read_dataset("time_sery.csv")

In [3]:
dataset.head(5)

,pluno,purchase_date,bndno,pl_1,pl_2,pl_3,pl_4,is_workday,qty,prev_d1,...,pl3_4week_min,pl4_2week_avg,pl4_2week_max,pl4_2week_min,pl4_3week_avg,pl4_3week_max,pl4_3week_min,pl4_4week_avg,pl4_4week_max,pl4_4week_min
0,22000005,2016-07-31,0,0.06,0.06,0.06,0.04,0,0.704,0.0,...,0.0,0.027893,1.070,0.0,0.023714,0.728,0.0,0.000000,0.000,0.0
1,22000008,2016-02-18,0,0.06,0.06,0.06,0.04,1,0.704,0.0,...,0.0,0.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.000,0.0
2,22000009,2016-07-25,0,0.06,0.06,0.06,0.04,1,0.666,0.0,...,0.0,0.023714,0.728,0.0,0.000000,0.000,0.0,0.056571,1.338,0.0
3,22000009,2016-07-27,0,0.06,0.06,0.06,0.04,1,1.120,0.0,...,0.0,0.032500,0.728,0.0,0.000000,0.000,0.0,0.023893,1.338,0.0
4,22000010,2016-03-16,0,0.06,0.06,0.06,0.04,1,1.914,0.0,...,0.0,0.036179,2.026,0.0,0.000000,0.000,0.0,0.012571,0.704,0.0


获得日期和商品订单的序号

In [4]:
plunos = np.unique(dataset['pluno'])
dates = np.unique(dataset['purchase_date'])

划分需要训练的数据和需要测试的数据,数据在Q2完成保存前被排过序

In [5]:
# 划分需要训练的数据和需要测试的数据,数据已经被预先排过序
train_dates = dates[:146]
test_dates = dates[146:]
# 用训练日期和测试日期获得对应的训练集和测试集
trainset=dataset[dataset['purchase_date'].isin(train_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
testset=dataset[dataset['purchase_date'].isin(test_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])

将pluno离散化化

In [6]:
precord = dict(dataset.loc[dataset["purchase_date"].isin(train_dates)].groupby(by=['pluno'])['qty'].mean())

查看训练集和测试集的起始日期

In [7]:
np.unique(trainset['pl_4'])

array([0.000e+00, 1.000e-02, 2.000e-02, 3.000e-02, 4.000e-02, 5.000e-02,
       6.000e-02, 7.000e-02, 8.000e-02, 9.000e-02, 1.000e-01, 1.600e-01,
       1.700e-01, 1.800e-01, 1.900e-01, 2.400e-01, 2.500e-01, 4.000e-01,
       4.500e-01, 4.600e-01, 6.600e-01, 7.800e-01, 9.000e-01, 1.090e+00,
       2.140e+00, 2.360e+00, 9.190e+00, 6.136e+01])

In [8]:
testset.head(5)

,pluno,purchase_date,bndno,pl_1,pl_2,pl_3,pl_4,is_workday,qty,prev_d1,...,pl3_4week_min,pl4_2week_avg,pl4_2week_max,pl4_2week_min,pl4_3week_avg,pl4_3week_max,pl4_3week_min,pl4_4week_avg,pl4_4week_max,pl4_4week_min
6389,22000005,2016-06-26,0,0.06,0.06,0.06,0.04,0,0.0,0.0,...,0.0,0.020286,1.136,0.0,0.042286,1.666,0.0,0.035679,1.308,0.0
6390,22000005,2016-06-27,0,0.06,0.06,0.06,0.04,1,0.0,0.0,...,0.0,0.037357,1.136,0.0,0.042286,1.666,0.0,0.035679,1.308,0.0
6391,22000005,2016-06-28,0,0.06,0.06,0.06,0.04,1,0.0,0.0,...,0.0,0.017071,0.956,0.0,0.062571,1.666,0.0,0.035679,1.308,0.0
6392,22000005,2016-06-29,0,0.06,0.06,0.06,0.04,1,0.0,0.0,...,0.0,0.017071,0.956,0.0,0.062571,1.666,0.0,0.035679,1.308,0.0
6393,22000005,2016-06-30,0,0.06,0.06,0.06,0.04,1,0.0,0.0,...,0.0,0.017071,0.956,0.0,0.062571,1.666,0.0,0.023357,1.308,0.0


因为训练集只是用来训练模型的,所以不需要保留时序数据,定义一个 $assemble\_training\_test$ 将其转化为 $data$ 和 $target$ 的形式

In [9]:
def assemble_training_set(trainset):
    
    temp = trainset.drop(['purchase_date', 'bndno'],axis=1)
    
    temp['pluno']=temp['pluno'].astype("int")
    
    target = temp['qty']
    temp = temp.drop(['qty'],axis = 1)
    

    temp['pl_1'] = temp['pl_1'].astype("float")
    temp['pl_2'] = temp['pl_2'].astype("float")
    temp['pl_3'] = temp['pl_3'].astype("float")
    temp['pl_4'] = temp['pl_4'].astype("float")
        
    data = {
        'data':temp.values,
        'target':target
    }
    return data

调用并检查训练数据维度

将测试集重新组织成用于更新时序数据的格式

In [10]:
def assemble_frame(df):
    initial_dataset = dict()
    temp = df.drop(['purchase_date','bndno'],axis=1)
    target = temp['qty']
    temp['pl_1'] = temp['pl_1'].astype("float")
    temp['pl_2'] = temp['pl_2'].astype("float")
    temp['pl_3'] = temp['pl_3'].astype("float")
    temp['pl_4'] = temp['pl_4'].astype("float")
    for pluno in plunos:
        temp_set = temp[temp['pluno']==pluno]
        temp_set['pluno']=[precord[x] for x in temp_set['pluno']]
        
        initial_dataset[pluno]={
            'data':temp_set.drop(['qty'],axis=1).values, 
            'target':temp_set['qty'].values
        }
    
    return initial_dataset

更新时序数据的函数,由于有多种场景,这里直接把要更新的特征所在的列号写死

In [11]:
def update_time_data(data,loc,pred,qty):
    if pred is None or qty is None:
        return
    elif pred<0:
        qty = qty - pred
    margin = min(35-loc,7)
    for scaner in range(margin):
        data[loc+scaner][6+scaner]=data[loc+scaner][6+scaner]+qty
    
    if data[loc].shape[0]==28:
        pass
#         data[loc][13] = data[loc][13]+qty/14
#         data[loc][16] = data[loc][16]+qty/21
#         data[loc][19] = data[loc][16]+qty/28
#         data[loc][14] = max(data[loc][14],pred)
#         data[loc][17] = max(data[loc][17],pred)
#         data[loc][20] = max(data[loc][20],pred)
#         data[loc][15] = min(data[loc][15],pred)
#         data[loc][18] = min(data[loc][18],pred)
#         data[loc][21] = min(data[loc][21],pred)
    elif data[loc].shape[0]==57:
        margin = min(35-loc,7)
        for scaner in range(margin):
            data[loc+scaner][13+scaner]=data[loc+scaner][13+scaner]+qty
            data[loc+scaner][20+scaner]=data[loc+scaner][20+scaner]+qty
            data[loc+scaner][27+scaner]=data[loc+scaner][27+scaner]+qty
            data[loc+scaner][34+scaner]=data[loc+scaner][34+scaner]+qty
            data[loc+scaner][41+scaner]=data[loc+scaner][41+scaner]+qty
        
#         data[loc][20] = data[loc][20]+qty/14
#         data[loc][23] = data[loc][23]+qty/21
#         data[loc][26] = data[loc][26]+qty/28
#         data[loc][21] = max(data[loc][21],pred)
#         data[loc][24] = max(data[loc][24],pred)
#         data[loc][27] = max(data[loc][27],pred)
#         data[loc][22] = min(data[loc][22],pred)
#         data[loc][25] = min(data[loc][25],pred)
#         data[loc][28] = min(data[loc][28],pred)
        
#         data[loc][29] = data[loc][29]+qty/14
#         data[loc][32] = data[loc][32]+qty/21
#         data[loc][35] = data[loc][35]+qty/28
#         data[loc][30] = max(data[loc][30],pred)
#         data[loc][33] = max(data[loc][33],pred)
#         data[loc][36] = max(data[loc][36],pred)
#         data[loc][31] = min(data[loc][31],pred)
#         data[loc][34] = min(data[loc][34],pred)
#         data[loc][37] = min(data[loc][37],pred)
        
    elif data[loc].shape[0]>57:
        margin = min(35-loc,7)
        for scaner in range(margin):
            data[loc+scaner][13+scaner]=data[loc+scaner][13+scaner]+qty
            data[loc+scaner][20+scaner]=data[loc+scaner][20+scaner]+qty
            data[loc+scaner][27+scaner]=data[loc+scaner][27+scaner]+qty
            data[loc+scaner][34+scaner]=data[loc+scaner][34+scaner]+qty
            data[loc+scaner][41+scaner]=data[loc+scaner][41+scaner]+qty
        
#         for i in range(6):
#             data[loc][48+i*9] = data[loc][48+i*9]+qty/14
#             data[loc][51+i*9] = data[loc][51+i*9]+qty/21
#             data[loc][54+i*9] = data[loc][54+i*9]+qty/28
#             data[loc][49+i*9] = max(data[loc][49+i*9],pred)
#             data[loc][52+i*9] = max(data[loc][52+i*9],pred)
#             data[loc][55+i*9] = max(data[loc][55+i*9],pred)
#             data[loc][50+i*9] = min(data[loc][50+i*9],pred)
#             data[loc][53+i*9] = min(data[loc][53+i*9],pred)
#             data[loc][56+i*9] = min(data[loc][56+i*9],pred)

用于计算每个pluno的 $RSE$, $sample\_rse$ 在论文中公式中的分子计算时用到, $sample\_base\_rse$ 在分母计算时用到

In [12]:
def sample_rse(data,target,loc,days,regressor):
    sample_rse = 0
    prev_pred,prev_qty = None, None
    #更新指定天数后得到qty
    for day in range(days):
        # 更新时序数据
        update_time_data(data,loc+day,prev_pred,prev_qty)
        
        predicted_target=regressor.predict(data[loc+day].reshape((1,-1)))[0]
        prev_pred = predicted_target
        real_target = target[loc+day]
        prev_qty = predicted_target-real_target
        #real_target = target[loc+day]
        
    sample_rse = prev_qty * prev_qty
    #print("predicted_target:",prev_pred)
    #sample_rse = sample_rse+sample_day_rse
    return sample_rse


def get_base_rse(test_set,day):
    total_pluno_qty = 0
    # 遍历每个商品
    for pluno in test_set.keys():
        total_pluno_qty = total_pluno_qty + sum_pluno_qty(test_set[pluno]['target'],day)
    
    # 算出分母里的平均
    avg_pluno_qty = total_pluno_qty / (len(test_set)*(len(test_dates)-7))
    
    total_base_rse = 0 
    for pluno in test_set.keys():
        total_base_rse = total_base_rse+ sum_pluno_qty((test_set[pluno]['target']-avg_pluno_qty)**2,day)
    return total_base_rse
    
def sum_pluno_qty(targets,day):
    sum_qty = 0
    length = targets.shape[0]
    return np.sum(targets[:-7])
#     for d in range(day):
#         sum_qty = sum_qty + np.sum(targets[day:length-6+day])

遍历测试集里的每一个pluno,累加最后获得 $RSE$

In [13]:

def calculate_rse(test_set,regressor,days):
    total_rse,base_rse = 0,0 
    for pluno in plunos:
        #表示7天
        test_data = copy.deepcopy(test_set[pluno]['data'])
        test_target = test_set[pluno]['target']
        for index in range(test_data.shape[0]-7):
            total_rse = total_rse+sample_rse(test_data,test_target,index,days,regressor)
    return total_rse
    #print(type(regressor).__name__)
    #print("RSE:",math.sqrt(total_rse)/math.sqrt(base_rse))

决策树,随机森林和支持向量机用 sklearn 的 api, mlp使用 pytorch 搭一个,可以添加dropout,设定层数,效果较好

In [14]:
# 决策树:
#tree_regressor = DecisionTreeRegressor()
# 随机森林
#forest_regressor = RandomForestRegressor(n_estimators=10)
# 支持向量机
svm_regressor = svm.SVR()
# mlp



在正式预测前,由于第四题设置了四个场景 (使用不同的特征),因此我们需要将这四个场景的特征线先划分到4个csv文件中。

## 1. 仅使用 b.i 特征

In [24]:
dataset = read_dataset("time_sery1.csv")
trainset=dataset.loc[dataset['purchase_date'].isin(train_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
testset=dataset.loc[dataset['purchase_date'].isin(test_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
train_data = assemble_training_set(trainset)
test_set = assemble_frame(testset)

C:\Users\Greilfang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### 1.1 随机森林

In [25]:
forest_regressor = RandomForestRegressor(n_estimators=50,max_depth=10)
forest_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)
for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,forest_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    print("day {} end:{}".format(d,result))
    step_rses.append(result)
print(np.mean(step_rses))

day 1 end:0.40805351726129135
day 2 end:1.0008101314161308
day 3 end:1.9191557751956771
day 4 end:4.958939396376208
day 5 end:5.190157361767456
day 6 end:5.44425663828467
day 7 end:5.566552534133916
3.498275050633621


In [ ]:
test_set

In [ ]:
step_rses

### 1.2 SVM

In [ ]:
svm_regressor = svm.SVR(C=0.5, kernel='rbf', gamma="auto",max_iter= 3000)
svm_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)

for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,svm_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    print("day {} end:{}".format(d,result))
    step_rses.append(result)
print(np.mean(step_rses))

In [ ]:
step_rses

### 1.3 MLP

In [22]:
mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(10,4),  activation='relu', solver='adam', alpha=0.0001, batch_size=2048,
    learning_rate='adaptive', learning_rate_init=0.0001, power_t=0.5, max_iter=200000, shuffle=True,
    tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False,beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)


for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,mlp_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    print("day {} end:{}".format(d,result))
    step_rses.append(result)
print(np.mean(step_rses))

day 1 end:0.8753102253624371
day 2 end:1.0080396334491428
day 3 end:1.038135666959623
day 4 end:1.4633611069002932
day 5 end:3.5756613840643765
day 6 end:10.014050310027283
day 7 end:23.56191727049414
5.933782228179614


## 2. 仅用 b.i 和 b.iv 特征

In [29]:
dataset = read_dataset("time_sery2.csv")
trainset=dataset[dataset['purchase_date'].isin(train_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
testset=dataset[dataset['purchase_date'].isin(test_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
train_data = assemble_training_set(trainset)
test_set = assemble_frame(testset)

C:\Users\Greilfang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### 2.1 决策树

In [30]:
# 这只是第七预测,第n天RSE要每一步前n步取平均
forest_regressor = RandomForestRegressor(n_estimators=20,max_depth=10)
forest_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)
for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,forest_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    print("day {} end:{}".format(d,result))
    step_rses.append(result)
print(np.mean(step_rses))

day 1 end:0.3657065660205173
day 2 end:1.73878018728672
day 3 end:2.51807215533027
day 4 end:2.640986799965096
day 5 end:3.4072071119471543
day 6 end:5.379465147646563
day 7 end:6.1403212885468035
3.1700770366775894


### 2.2 SVM

In [31]:
svm_regressor = svm.SVR(C=0.5, kernel='rbf', gamma="auto",max_iter= 3000)
svm_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)

for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,svm_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    step_rses.append(result)
    print("day {} end:{}".format(d,np.mean(step_rses)))
print(np.mean(step_rses))

C:\Users\Greilfang\Anaconda3\lib\site-packages\sklearn\svm\base.py:244: ConvergenceWarning: Solver terminated early (max_iter=7000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


KeyboardInterrupt: 

### 2.3 MLP

In [32]:
mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(18,8),  activation='relu', solver='adam', alpha=0.0001, batch_size=2048,
    learning_rate='adaptive', learning_rate_init=0.0001, power_t=0.5, max_iter=200000, shuffle=True,
    tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False,beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)


for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,mlp_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    step_rses.append(result)
    print("day {} end:{}".format(d,np.mean(step_rses)))
print(np.mean(step_rses))

day 1 end:0.6943634128428352
day 2 end:0.7052097775473407
day 3 end:0.7462560571712302
day 4 end:0.8100420205858041
day 5 end:0.8794033106011743
day 6 end:0.969654960735058
day 7 end:1.104140667681129
1.104140667681129


## 3. 仅适用 b.i + b.iv + b.ii +b.v  特征

In [33]:
dataset = read_dataset("time_sery3.csv")
trainset=dataset[dataset['purchase_date'].isin(train_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
testset=dataset[dataset['purchase_date'].isin(test_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
train_data = assemble_training_set(trainset)
test_set = assemble_frame(testset)

C:\Users\Greilfang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### 3.1 随机森林

In [34]:
forest_regressor = RandomForestRegressor(n_estimators=30,max_depth=10)
forest_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)
for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,forest_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    print("day {} end:{}".format(d,result))
    step_rses.append(result)
print(np.mean(step_rses))

day 1 end:0.33420506303244557
day 2 end:1.6459634217129648
day 3 end:2.7472183800059216
day 4 end:3.064354979006559
day 5 end:4.2352454890483555
day 6 end:5.258971080678135
day 7 end:6.032580510166506
3.3312198462358404


### 3.2 SVM

In [ ]:
svm_regressor = svm.SVR(C=0.5, kernel='rbf', gamma="auto",max_iter= 3000)
svm_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)

for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,svm_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    step_rses.append(result)
    print("day {} end:{}".format(d,np.mean(step_rses)))
print(np.mean(step_rses))

### 3.3 MLP

In [35]:
mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(32,16),  activation='relu', solver='adam', alpha=0.0001, batch_size=2048,
    learning_rate='adaptive', learning_rate_init=0.0001, power_t=0.5, max_iter=200000, shuffle=True,
    tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False,beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)


for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,mlp_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    step_rses.append(result)
    print("day {} end:{}".format(d,np.mean(step_rses)))
print(np.mean(step_rses))

day 1 end:0.7557151225540063
day 2 end:0.8272161052089024
day 3 end:0.9500560536559851
day 4 end:1.0877422862111317
day 5 end:1.219521475585465
day 6 end:1.3495092405738731
day 7 end:1.4509932956450324
1.4509932956450324


## 4. 使用全部特征

In [36]:
train_dates = dates[21:]
dataset = read_dataset("time_sery4.csv")
trainset=dataset[dataset['purchase_date'].isin(train_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
testset=dataset[dataset['purchase_date'].isin(test_dates)].sort_values(by=['pluno','purchase_date'],ascending=[True,True])
train_data = assemble_training_set(trainset)
test_set = assemble_frame(testset)

C:\Users\Greilfang\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


### 4.1 随机森林

In [38]:
forest_regressor = RandomForestRegressor(n_estimators=30,max_depth=10)
forest_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)
for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,forest_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    step_rses.append(result)
    print("day {} end:{}".format(d,np.mean(step_rses)))
print(np.mean(step_rses))

day 1 end:0.22082948682220793
day 2 end:0.6435019340119283
day 3 end:1.305742041530783
day 4 end:1.723975827999527
day 5 end:2.1146697008462256
day 6 end:2.487461731852385
day 7 end:2.856799972387743
2.856799972387743


### 4.3 SVM

In [ ]:
svm_regressor = svm.SVR(C=0.5, kernel='rbf', gamma="auto",max_iter= 3000)
svm_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)

for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,svm_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    step_rses.append(result)
    print("day {} end:{}".format(d,np.mean(step_rses)))
print(np.mean(step_rses))

### 4.4 MLP

In [56]:
mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(64,36,16),  activation='relu', solver='adam', alpha=0.0001, batch_size=2048,
    learning_rate='adaptive', learning_rate_init=0.0001, power_t=0.5, max_iter=200000, shuffle=True,
    tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False,beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp_regressor.fit(train_data['data'],train_data['target'])
step_rses = list()
base_rse = get_base_rse(test_set,7)


for d in range(1,7+1):
    predict_rse=calculate_rse(test_set,mlp_regressor,d)
    result = math.sqrt(predict_rse)/math.sqrt(base_rse)
    step_rses.append(result)
    print("day {} end:{}".format(d,np.mean(step_rses)))
print(np.mean(step_rses))

C:\Users\Greilfang\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


day 1 end:1.2986071736493512
day 2 end:1.3125264461807593
day 3 end:1.349650520876328
day 4 end:1.398295478745645
day 5 end:1.4396624848212738
day 6 end:1.4882207912565493
day 7 end:1.5352600480228646
2.302890072034297
